In [2]:
from pynq import Overlay
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
from os import path
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone

from TestChip import TestChip

%matplotlib inline

# Experiment 1 

Collect grid data of RO frequency.

# Experiment 2
Collect randomply placed RO frequency data on 160 ROs.

In [9]:
def record(ol, total_duration, every, num_oscillators):
    """
    parameters
    ----------
    total_duration : total duration in seconds
    every : sampling step size in seconds
    """
    data = []   # [RO0, ..., RO{num_oscillators},Temperature, En_Freq, Duty_Cycle]
    RO_dict = dict(RO0=list(range(32)), RO1=list(range(32)), RO2=list(range(32)), RO3=list(range(32)))
    times = []
    init_time = datetime.now()
    now_time = init_time
    while(now_time < (init_time + total_duration)):
        now_time = datetime.now()
        temp = ol.XADC_temp()
        output_dict = ol.read_multi_RO(RO_dict)
        current_read = np.hstack((output_dict['RO0'], output_dict['RO1']))
        current_read = np.hstack((current_read, output_dict['RO2']))
        current_read = np.hstack((current_read, output_dict['RO3']))
        current_read = np.hstack((current_read, np.array([temp, 0, 0])))
        times.append(now_time)
        data.append(current_read)
        while(datetime.now() < now_time + every):
            pass
        print(now_time)
    data = np.vstack(data)
    output = pd.DataFrame(data, columns=([f'RO{i}' for i in range(num_oscillators)] + ['Temperature', 'En_Freq', 'Duty_Cycle']))
    output['Timestamp'] = pd.DataFrame(dict(Timestamp=times))
    return output

In [10]:
every = timedelta(seconds = 5)
total_duration = timedelta(seconds = 7)

for i in range(0, 7):
    ol = TestChip(f'/home/xilinx/pynq/overlays/RO/RO_random{i}.bit')
    output = record(ol, total_duration, every, 128)
    output.to_pickle(f'././ALL_3_Stage_128_Osc_rand_{i}.pkl')

2021-05-07 17:04:53.367331
2021-05-07 17:04:58.367873
2021-05-07 17:05:03.368379
2021-05-07 17:05:08.368890
2021-05-07 17:05:13.370238
2021-05-07 17:05:18.910177
2021-05-07 17:05:23.910680
2021-05-07 17:05:28.911210
2021-05-07 17:05:33.911730
2021-05-07 17:05:38.912246
2021-05-07 17:05:44.400264
2021-05-07 17:05:49.400783
2021-05-07 17:05:54.401323
2021-05-07 17:05:59.401822
2021-05-07 17:06:04.402338
2021-05-07 17:06:09.890577
2021-05-07 17:06:14.891203
2021-05-07 17:06:19.891706
2021-05-07 17:06:24.892211
2021-05-07 17:06:29.892713
2021-05-07 17:06:35.383202
2021-05-07 17:06:40.383710
2021-05-07 17:06:45.384229
2021-05-07 17:06:50.384729
2021-05-07 17:06:55.385232
2021-05-07 17:07:01.202438
2021-05-07 17:07:06.202943
2021-05-07 17:07:11.203462
2021-05-07 17:07:16.203960
2021-05-07 17:07:21.204476
2021-05-07 17:07:26.689903
2021-05-07 17:07:31.690420
2021-05-07 17:07:36.690966
2021-05-07 17:07:41.691469
2021-05-07 17:07:46.692749


In [11]:
output.head()

,RO0,RO1,RO2,RO3,RO4,RO5,RO6,RO7,RO8,RO9,...,RO122,RO123,RO124,RO125,RO126,RO127,Temperature,En_Freq,Duty_Cycle,Timestamp
0,400.286,399.817,430.227,428.501,414.691,439.404,406.328,431.009,423.391,399.470,...,441.260,433.283,434.867,428.456,426.500,425.822,52.415881,0.0,0.0,2021-05-07 17:07:26.689903
1,400.362,399.892,430.091,428.560,414.838,439.259,406.181,430.762,423.050,399.292,...,441.126,433.298,434.651,428.362,426.473,425.566,53.154126,0.0,0.0,2021-05-07 17:07:31.690420
2,400.377,399.703,429.930,428.390,414.553,439.248,406.210,430.841,423.204,399.371,...,441.176,433.262,434.600,428.250,426.519,425.619,53.277167,0.0,0.0,2021-05-07 17:07:36.690966
3,400.423,399.802,430.183,428.207,414.480,439.203,406.238,430.673,423.094,399.300,...,441.307,433.488,434.696,428.391,426.434,425.444,53.277167,0.0,0.0,2021-05-07 17:07:41.691469
4,400.241,399.661,430.038,428.370,414.734,439.270,406.355,430.815,423.171,399.396,...,441.248,433.301,434.659,428.268,426.479,425.528,53.400208,0.0,0.0,2021-05-07 17:07:46.692749
